In [4]:
import codecs
import os
import shutil
import numpy as np

#Open train dev and test .txt and read lines

#TRAIN
traintxt = codecs.open('../subview_train.txt', 'r', 'utf-8').read()
#remove description
traintxt = traintxt[traintxt.index(',') + 1:]
#string to list
trainids = traintxt.split(",")
print("Anzahl TrainingsIds: ", len(trainids))

#DEV
devtxt = codecs.open('../subview_dev.txt', 'r', 'utf-8').read()
#remove description
devtxt = devtxt[devtxt.index(',') + 1:]
#string to list
devids = devtxt.split(",")
print("Anzahl DevIds: ", len(devids))

#TEST
testtxt = codecs.open('../subview_test.txt', 'r', 'utf-8').read()
#remove description
testtxt = testtxt[testtxt.index(',') + 1:]
#string to list
testids = testtxt.split(",")
print("Anzahl TestIds: ", len(testids))



Anzahl TrainingsIds:  507107
matching_utterance_ids
include,00000019_0000000025_101370_103180,00000019_0000000025_104560_107260,0
Anzahl DevIds:  22555
['00000019_0000000025_101370_103180', '00000019_0000000025_104560_107260', '00000019_0000000025_109370_117330', '00000019_0000000025_121710_123410', '00000019_0000000025_125330_127530']
Anzahl TestIds:  22287


In [9]:
trainnumber = 0
devnumber = 0 
testnumber = 0
deletenumber = 0

foundtestids = []
foundtrainids = []
founddevids = []
founddeleteids = []

# traverse root directory move dev and test from trainset, delete unused (unused because same audio, different microphone)
#get and sort dirs
for outerroot, outerdirs, outerfilenames in os.walk(u"./data/train"):
    outerdirs.sort()
    #for every dir use current, non recursive Files only
    for dirname in outerdirs:
        dirpath = os.path.join(outerroot, dirname)
        print(dirpath)
        for root, dirs, filenames in os.walk(dirpath):
            for filename in filenames:
                #print(os.path.abspath(os.path.join(root, filename)))
                if filename.endswith(".trans.txt"):
                    print("filename: ", filename)
                    trans_filepath = os.path.abspath(os.path.join(root, filename))
                    #print("trans_filepath: ", trans_filepath)
                    folder2path = os.path.dirname(trans_filepath)
                    #print("folder2path: ", folder2path)
                    folder2name = folder2path.split("/")[-1].split("\\")[-1]
                    #print("folder2name: ", folder2name)
                    folder1path = os.path.dirname(folder2path)
                    folder1name = folder1path.split("/")[-1].split("\\")[-1]
                    trainpath = os.path.dirname(folder1path)
                    datapath = os.path.dirname(trainpath)
                    devpath = datapath + "/dev"
                    testpath = datapath + "/test"
                    newtrain_transtxtLines = []
                    newdev_transtxtLines = []
                    newtest_transtxtLines = []
                    newtrainPath = datapath + "/train" + "/" + folder1name + "/" + folder2name
                    newdevPath = datapath + "/dev"   + "/" + folder1name + "/" + folder2name
                    newtestPath = datapath + "/test"  + "/" + folder1name + "/" + folder2name

                    with codecs.open(trans_filepath, 'r') as file:
                        transtxtLines = file.read().splitlines()
                        for l in transtxtLines:
                            #skip empty lines
                            if len(l) > 0:
                                seq_name, txt = l.split(" ", 1)
                                seq_path = os.path.abspath(os.path.join(root, seq_name))
                                seq_id = seq_name.split("-", 2)[-1]#.split(".wav")[0]
                                #print (seq_name)
                                #print(txt)
                                #print(seq_id)
                                if seq_id in trainids:
                                    #print("TRAIN")
                                    trainnumber += 1
                                    foundtrainids.append(seq_id)
                                    newtrain_transtxtLines.append(l)
                                    #file not need to be moved
                                    if trainnumber % 10000 == 0:
                                        print("trainnumber: ", trainnumber)
                                elif seq_id in devids:
                                    #print("DEV")
                                    devnumber += 1
                                    founddevids.append(seq_id)
                                    newdev_transtxtLines.append(l)
                                    os.makedirs(newdevPath, exist_ok=True)
                                    if os.path.isfile(seq_path):
                                        shutil.move(seq_path, newdevPath + "/" + seq_name)
                                    if devnumber % 10000 == 0:
                                        print("devnumber: ", devnumber)
                                elif seq_id in testids:
                                    #print("TEST")
                                    testnumber += 1
                                    foundtestids.append(seq_id)
                                    newtest_transtxtLines.append(l)
                                    os.makedirs(newtestPath, exist_ok=True)
                                    if os.path.isfile(seq_path):
                                        shutil.move(seq_path, newtestPath + "/" + seq_name)
                                    if testnumber % 10000 == 0:
                                        print("testnumber: ", testnumber)
                                else:
                                    #print("DELETE")
                                    deletenumber += 1
                                    founddeleteids.append(seq_id)
                                    if os.path.isfile(seq_path):
                                        os.remove(seq_path)
                                    if deletenumber % 10000 == 0:
                                        print("deletenumber: ", deletenumber)

                    #write new txt lines to txt files

                    #train
                    #folder already exists, traintrans.txt can be overwritten no matter what!
                    
                    with codecs.open(newtrainPath + "/" + filename, 'w') as f:
                        for item in newtrain_transtxtLines:
                            f.write("%s\n" % item)
                    
                    #dev
                    
                    if len(newdev_transtxtLines) > 0:
                        with codecs.open(newdevPath + "/" + filename, 'w') as f:
                            for item in newdev_transtxtLines:
                                f.write("%s\n" % item) 
                    
                    #test
                    
                    if len(newtest_transtxtLines) > 0:
                        with codecs.open(newtestPath + "/" + filename, 'w') as f:
                            for item in newtest_transtxtLines:
                                f.write("%s\n" % item)
                    
            ##stop before dirs, iterate current files only
            break
    

        

                
                

print("Anzahl Trainingssamples found: ", trainnumber)
print("Anzahl Devsamples found: ", devnumber)
print("Anzahl Testsamples found: ", testnumber)
print("Anzahl Unsuedsamples found: ", deletenumber)

#check what wasnt found
print("Missing Trainlabels: ", len(trainids) - len(foundtrainids))
missing_train_labels = np.setdiff1d(trainids, foundtrainids)
print("tranisids: ", len(trainids))
print("foundtrainids: ", len(foundtrainids))
print("missing_train_labels: ", len(missing_train_labels))
if len(missing_train_labels) > 100:
    print(missing_train_labels[:100])
else:
    print(missing_train_labels)

print("Missing Devlabels: ", len(devids) - len(founddevids))
missing_dev_labels = np.setdiff1d(devids, founddevids)
if len(missing_dev_labels) > 100:
    print(missing_dev_labels[:100])
else:
    print(missing_dev_labels)
    
print("Missing Testlabels: ", len(testids) - len(foundtestids))
missing_test_labels = np.setdiff1d(testids, foundtestids)
if len(missing_test_labels) > 100:
    print(missing_test_labels[:100])
else:
    print(missing_test_labels)


./data/train\0
./data/train\1
./data/train\2
./data/train\3
./data/train\4
./data/train\5
./data/train\6
./data/train\7
./data/train\8
./data/train\9
./data/train\0\0
filename:  0-0.trans.txt
trainnumber:  10000
trainnumber:  20000
./data/train\0\1
filename:  0-1.trans.txt
trainnumber:  30000
trainnumber:  40000
./data/train\0\2
filename:  0-2.trans.txt
trainnumber:  50000
trainnumber:  60000
trainnumber:  70000
./data/train\0\3
filename:  0-3.trans.txt
trainnumber:  80000
trainnumber:  90000
./data/train\1\4
filename:  1-4.trans.txt
trainnumber:  100000
trainnumber:  110000
trainnumber:  120000
./data/train\1\5
filename:  1-5.trans.txt
trainnumber:  130000
trainnumber:  140000
./data/train\1\6
filename:  1-6.trans.txt
trainnumber:  150000
trainnumber:  160000
./data/train\1\7
filename:  1-7.trans.txt
trainnumber:  170000
trainnumber:  180000
trainnumber:  190000
./data/train\2\10
filename:  2-10.trans.txt
trainnumber:  200000
trainnumber:  210000
./data/train\2\11
filename:  2-11.tran

In [80]:

list_1=["a", "b", "c", "d", "e"]
list_2=["a"]


import numpy as np
np.setdiff1d(list_1,list_2)

array(['b', 'c', 'd', 'e'], dtype='<U1')

In [6]:
import os
root="C:\\Users\johan\\Desktop\\megs_splittest\\full_waverized\\librispeechformat"
l= "3-14-AndreyBarrientosCostaRica-20130709-cjw-de11-032 BEIM TIEFBAU WIRD DAS KABEL EINFACH AUF DEM BODEN AUSGELEGT"
seq_id, txt = l.split(" ", 1)
seq_path = os.path.abspath(os.path.join(root, seq_id))

print(seq_id)
print(txt)
print(seq_path)


3-14-AndreyBarrientosCostaRica-20130709-cjw-de11-032
BEIM TIEFBAU WIRD DAS KABEL EINFACH AUF DEM BODEN AUSGELEGT
C:\Users\johan\Desktop\megs_splittest\full_waverized\librispeechformat\3-14-AndreyBarrientosCostaRica-20130709-cjw-de11-032


In [18]:
id = "00000019_0000000025_101370_103180"
id = "00000018_0000000024_999030_1003020"

#durchsuche TXTs nach id
for outerroot, outerdirs, outerfilenames in os.walk(u"./data/train"):
    outerdirs.sort()
    #for every dir use current, non recursive Files only
    for dirname in outerdirs:
        dirpath = os.path.join(outerroot, dirname)
        print(dirpath)
        for root, dirs, filenames in os.walk(dirpath):
            for filename in filenames:
                #print(os.path.abspath(os.path.join(root, filename)))
                if filename.endswith(".trans.txt"):
                    #print("filename: ", filename)
                    trans_filepath = os.path.abspath(os.path.join(root, filename))
                    with codecs.open(trans_filepath, 'r') as file:
                        transtxtlines = file.read().splitlines()
                        for line in transtxtlines:
                            if id in line:
                                print("Id found in: ", filename)

./data/train\0
Id found in:  0-1.trans.txt
./data/train\1
./data/train\2
./data/train\3
./data/train\4
./data/train\5
./data/train\6
./data/train\7
./data/train\8
./data/train\9
./data/train\0\0
./data/train\0\1
Id found in:  0-1.trans.txt
./data/train\0\2
./data/train\0\3
./data/train\1\4
./data/train\1\5
./data/train\1\6
./data/train\1\7
./data/train\2\10
./data/train\2\11
./data/train\2\8
./data/train\2\9
./data/train\3\12
./data/train\3\13
./data/train\3\14
./data/train\3\15
./data/train\4\16
./data/train\4\17
./data/train\4\18
./data/train\4\19
./data/train\5\20
./data/train\5\21
./data/train\5\22
./data/train\5\23
./data/train\6\24
./data/train\6\25
./data/train\6\26
./data/train\6\27
./data/train\7\28
./data/train\7\29
./data/train\7\30
./data/train\7\31
./data/train\8\32
./data/train\8\33
./data/train\8\34
./data/train\8\35
./data/train\9\36
